In [1]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd

from deploy_final import TradingSystem, PerformanceTracker, FastDataManager

ccys= [
            'GBP_CHF', 'GBP_JPY', 'EUR_CHF', 'EUR_JPY', 'USD_CHF',
            'EUR_CAD', 'EUR_USD', 'GBP_USD', 'EUR_GBP', 'USD_JPY',
            'USD_CAD', 'AUD_USD', 'CHF_JPY', 'AUD_JPY', 'NZD_USD',
            'NZD_JPY', 'XAU_USD', 'XAG_USD'
        ]

trading_system = TradingSystem()
# trading_system.run()
# trading_system.initialize()
# data_manager = FastDataManager()
# data_manager.initialize_pair('EUR_USD')

data_manager = FastDataManager(
    base_storage_path="./raw_data"
)

# Update all raw data files
update_results = data_manager.update_all_raw_data()

# Shutdown properly
# data_manager.shutdown()

# for ccy in ccys:
#     load_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/to_test_deploy/{ccy}_5T_indics_1H_not_norm.parquet'
#     save_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/to_test_deploy/{ccy}_raw_5min.parquet'
#     df = pd.read_parquet(load_path)
#     df = df.tail(200_000)
#     df = df[['open', 'high', 'low', 'close']]
#     df.to_parquet(save_path)

Logging configuration loaded from /Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/config/logging_config.yaml
2024-12-04 10:21:44 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:207 - setting up API-client for environment practice
2024-12-04 10:21:44 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:207 - setting up API-client for environment practice
2024-12-04 10:21:44 - trading_system - INFO - deploy_final.py:1131 - Starting update for EUR_USD
2024-12-04 10:21:44 - trading_system - INFO - deploy_final.py:1155 - Fetching new data for EUR_USD since 2024-12-04 09:05:00+00:00
2024-12-04 10:21:44 - trading_system - INFO - deploy_final.py:787 - Fetching missing candles for EUR_USD - time 2024-12-04 09:21:44
2024-12-04 10:21:44 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:261 - performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
2024-12-04 10:21:44 - trading_system - INFO - deploy_final.py:810 - CANDLES for EUR_USD - time: 2024-12-04 09

In [3]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import oandapyV20.endpoints.instruments as instruments
from oandapyV20 import API
import pandas as pd
# OANDA Configuration
OANDA_API_KEY = '9317ace4596d61e3e98b1a53b2342483-45d3ad4084c80b111727a9fada9ef0ff'
OANDA_ACCOUNT_ID = '101-004-30348600-001'  # running account
# OANDA_ACCOUNT_ID = '101-004-30348600-002'
OANDA_ENV = 'practice'

# Initialize OANDA client
client = API(access_token=OANDA_API_KEY, environment=OANDA_ENV)

current_raw_data = pd.read_parquet('./raw_data/XAG_USD_raw_5min.parquet')
current_time = pd.Timestamp.now(tz='UTC')
print(f'time before fetching runs: {current_time}')
last_timestamp = current_raw_data.index[-1]
print(f'last_timestamp: {last_timestamp}')


def fetch_missing_candles(pair: str, last_timestamp: pd.Timestamp) -> pd.DataFrame:
        """
        Fetches new candles from OANDA with smart handling of incomplete candles.

        Strategy:
        1. Calculate the expected candle timestamps based on 5-min intervals
        2. Fetch both complete and incomplete candles
        3. Use incomplete candles only when they're close to completion
        4. Implement safety checks for data quality
        """

        try:
            # Calculate the expected current candle timestamp (rounded down to 5-min intervals)
            current_time = pd.Timestamp.now(tz='UTC')
            print(
                f'Fetching missing candles for {pair} - time {current_time}')
            expected_candle_time = current_time.floor('5min')
            print(f'expected_candle_time: {expected_candle_time}')

            # Set up parameters for OANDA request
            params = {
                "from": last_timestamp.strftime('%Y-%m-%dT%H:%M:%SZ'),
                "granularity": "M5",
                "price": "M"
            }

            r = instruments.InstrumentsCandles(instrument=pair, params=params)
            response = client.request(r)
            candles = response.get('candles', [])

            if not candles:
                return pd.DataFrame()

            df_list = []
            print(f'CANDLES for {pair} - time: {current_time}')
            print('-'*50)
            print(candles)
            print('-'*50)
            for candle in candles:
                candle_time = pd.to_datetime(candle['time'], utc=True)

                # Logic for handling different candle scenarios
                should_include = False

                if candle['complete']:
                    # Always include complete candles
                    should_include = True
                else:
                    # For incomplete candles, check if it's the current expected candle
                    if candle_time == expected_candle_time:
                        # Calculate how far into the 5-min interval we are
                        elapsed_time = (
                            current_time - candle_time).total_seconds()
                        print(f'candle elapsed time: {elapsed_time}')

                        # Include if we're at least 4.5 minutes into the candle
                        # This means the candle is 90% complete
                        if elapsed_time >= 270:  # 4.5 minutes = 270 seconds
                            should_include = True
                            print(f"Including incomplete candle for {pair} at {candle_time} "
                                        f"({elapsed_time:.1f} seconds elapsed)")

                if should_include:
                    df_list.append({
                        'timestamp': candle_time,
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        # Add flag for tracking
                        'is_complete': candle['complete']
                    })

            if not df_list:
                return pd.DataFrame()

            df = pd.DataFrame(df_list)
            df.set_index('timestamp', inplace=True)

            # Log information about the fetched data
            print(
                f"Fetched {len(df)} candles for {pair} at time {pd.Timestamp.now(tz='UTC')}")
            print(f'Last candle timestamp for {pair}: {df.index[-1]}')
            print(df)

            incomplete_count = len(df[~df['is_complete']])
            if incomplete_count > 0:
                print(
                    f"Included {incomplete_count} incomplete candles for {pair}")

            return df

        except Exception as e:
            print(f"Error fetching candles for {pair}: {str(e)}")
            return pd.DataFrame()

fetch_missing_candles('XAG_USD', last_timestamp)

2024-12-04 10:57:34 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:207 - setting up API-client for environment practice
time before fetching runs: 2024-12-04 09:57:34.904518+00:00
last_timestamp: 2024-12-04 09:15:00+00:00
Fetching missing candles for XAG_USD - time 2024-12-04 09:57:34.907145+00:00
expected_candle_time: 2024-12-04 09:55:00+00:00
2024-12-04 10:57:34 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:261 - performing request https://api-fxpractice.oanda.com/v3/instruments/XAG_USD/candles
CANDLES for XAG_USD - time: 2024-12-04 09:57:34.907145+00:00
--------------------------------------------------
[{'complete': True, 'volume': 204, 'time': '2024-12-04T09:15:00.000000000Z', 'mid': {'o': '30.85900', 'h': '30.86350', 'l': '30.82300', 'c': '30.83200'}}, {'complete': True, 'volume': 166, 'time': '2024-12-04T09:20:00.000000000Z', 'mid': {'o': '30.82900', 'h': '30.85700', 'l': '30.82600', 'c': '30.84850'}}, {'complete': True, 'volume': 137, 'time': '2024-12-04T09:25:00.000000000Z', 

,open,high,low,close,is_complete
timestamp,,,,,
2024-12-04 09:15:00+00:00,30.8590,30.8635,30.8230,30.8320,True
2024-12-04 09:20:00+00:00,30.8290,30.8570,30.8260,30.8485,True
2024-12-04 09:25:00+00:00,30.8430,30.8650,30.8235,30.8575,True
2024-12-04 09:30:00+00:00,30.8570,30.8635,30.8300,30.8300,True
2024-12-04 09:35:00+00:00,30.8305,30.8425,30.8160,30.8335,True
2024-12-04 09:40:00+00:00,30.8330,30.8525,30.8160,30.8170,True
2024-12-04 09:45:00+00:00,30.8160,30.8280,30.7980,30.8025,True
2024-12-04 09:50:00+00:00,30.8020,30.8330,30.7955,30.8220,True


In [6]:
ccy = 'EUR_CAD'
save_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/to_test_deploy/{ccy}_raw_5min.parquet'
df = pd.read_parquet(save_path)
df

,open,high,low,close
timestamp,,,,
2022-03-15 08:20:00+00:00,1.41360,1.41500,1.41360,1.41480
2022-03-15 08:25:00+00:00,1.41480,1.41480,1.41430,1.41430
2022-03-15 08:30:00+00:00,1.41420,1.41470,1.41400,1.41470
2022-03-15 08:35:00+00:00,1.41460,1.41530,1.41450,1.41510
2022-03-15 08:40:00+00:00,1.41520,1.41530,1.41480,1.41490
...,...,...,...,...
2024-12-03 15:00:00+00:00,1.47717,1.47731,1.47593,1.47628
2024-12-03 15:05:00+00:00,1.47630,1.47671,1.47627,1.47646
2024-12-03 15:10:00+00:00,1.47648,1.47682,1.47626,1.47674


In [1]:
import pandas as pd
eur = pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/to_test_deploy/EUR_USD_5T_indics_1H_not_norm.parquet')
eur

,open,high,low,close,sma_20,sma_50,rsi,macd,macd_signal,macd_hist,...,bb_bandwidth,bb_percent,atr,plus_di,minus_di,adx,senkou_span_a,senkou_span_b,tenkan_sen,kijun_sen
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-25 00:00:00+00:00,1.05930,1.05960,1.05930,1.05960,1.056195,1.063512,46.528531,-0.001022,-0.002158,0.001136,...,1.646508,47.153591,0.006270,23.247270,18.182049,19.290074,1.070725,1.0819,1.060400,1.056850
2023-10-25 00:05:00+00:00,1.05950,1.05970,1.05950,1.05960,1.056195,1.063512,46.528531,-0.001022,-0.002158,0.001136,...,1.646508,47.153591,0.006270,23.247270,18.182049,19.290074,1.070725,1.0819,1.060400,1.056850
2023-10-25 00:10:00+00:00,1.05960,1.05960,1.05950,1.05960,1.056195,1.063512,46.528531,-0.001022,-0.002158,0.001136,...,1.646508,47.153591,0.006270,23.247270,18.182049,19.290074,1.070725,1.0819,1.060400,1.056850
2023-10-25 00:15:00+00:00,1.05960,1.05960,1.05950,1.05950,1.056195,1.063512,46.528531,-0.001022,-0.002158,0.001136,...,1.646508,47.153591,0.006270,23.247270,18.182049,19.290074,1.070725,1.0819,1.060400,1.056850
2023-10-25 00:20:00+00:00,1.05960,1.05970,1.05960,1.05970,1.056195,1.063512,46.528531,-0.001022,-0.002158,0.001136,...,1.646508,47.153591,0.006270,23.247270,18.182049,19.290074,1.070725,1.0819,1.060400,1.056850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03 14:45:00+00:00,1.05208,1.05252,1.05202,1.05204,1.053768,1.072592,40.218558,-0.006334,-0.007621,0.001287,...,2.232796,36.792599,0.007616,12.444874,28.465143,41.434634,1.086720,1.0986,1.051115,1.063465
2024-12-03 14:50:00+00:00,1.05204,1.05210,1.05171,1.05208,1.053768,1.072592,40.218558,-0.006334,-0.007621,0.001287,...,2.232796,36.792599,0.007616,12.444874,28.465143,41.434634,1.086720,1.0986,1.051115,1.063465
2024-12-03 14:55:00+00:00,1.05207,1.05215,1.05174,1.05198,1.053768,1.072592,40.218558,-0.006334,-0.007621,0.001287,...,2.232796,36.792599,0.007616,12.444874,28.465143,41.434634,1.086720,1.0986,1.051115,1.063465


In [ ]:
import pandas as pd
jpy = pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/to_test_deploy/USD_JPY_5T_indics_1H_not_norm.parquet')
jpy


,open,high,low,close,sma_20,sma_50,rsi,macd,macd_signal,macd_hist,...,bb_bandwidth,bb_percent,atr,plus_di,minus_di,adx,senkou_span_a,senkou_span_b,tenkan_sen,kijun_sen
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-01-08 05:00:00+00:00,116.290,116.290,116.260,116.260,116.37500,115.72180,56.114757,0.145339,0.205676,-0.060337,...,0.510566,44.109447,0.181639,24.736054,18.461754,35.768578,115.68750,115.200,116.435,116.3600
2001-01-08 05:05:00+00:00,116.260,116.300,116.260,116.300,116.37500,115.72180,56.114757,0.145339,0.205676,-0.060337,...,0.510566,44.109447,0.181639,24.736054,18.461754,35.768578,115.68750,115.200,116.435,116.3600
2001-01-08 05:10:00+00:00,116.300,116.310,116.300,116.310,116.37500,115.72180,56.114757,0.145339,0.205676,-0.060337,...,0.510566,44.109447,0.181639,24.736054,18.461754,35.768578,115.68750,115.200,116.435,116.3600
2001-01-08 05:15:00+00:00,116.310,116.340,116.310,116.340,116.37500,115.72180,56.114757,0.145339,0.205676,-0.060337,...,0.510566,44.109447,0.181639,24.736054,18.461754,35.768578,115.68750,115.200,116.435,116.3600
2001-01-08 05:20:00+00:00,116.340,116.340,116.290,116.290,116.37500,115.72180,56.114757,0.145339,0.205676,-0.060337,...,0.510566,44.109447,0.181639,24.736054,18.461754,35.768578,115.68750,115.200,116.435,116.3600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-28 09:55:00+00:00,151.882,151.892,151.845,151.892,151.33680,152.19464,55.742599,-0.073980,-0.212951,0.138971,...,0.933599,89.295656,0.291988,22.328450,19.090376,23.460515,152.78600,153.284,151.646,151.2075
2024-11-28 10:00:00+00:00,151.892,151.899,151.862,151.875,151.36475,152.15390,56.743652,-0.042292,-0.178819,0.136527,...,0.994521,87.682177,0.277203,21.839402,18.672250,22.343184,152.52925,153.078,151.646,151.2025
2024-11-28 10:05:00+00:00,151.876,151.944,151.874,151.928,151.36475,152.15390,56.743652,-0.042292,-0.178819,0.136527,...,0.994521,87.682177,0.277203,21.839402,18.672250,22.343184,152.52925,153.078,151.646,151.2025


In [ ]:
# Add the project root to the Python path
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from data_management.preprocessor import DataPreprocessor
from data_management.indicator_manager import IndicatorManager
from typing import Optional, Tuple
from stable_baselines3.common.vec_env import VecNormalize
import oandapyV20.endpoints.accounts as accounts
from typing import Dict, Optional
from dataclasses import dataclass
import talib
from dataclasses import dataclass, field
import pytz
from typing import Dict, List, Optional, Tuple
from google.cloud import storage
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from oandapyV20.endpoints.positions import OpenPositions
from oandapyV20.endpoints.transactions import TransactionsSinceID
import oandapyV20.endpoints.instruments as instruments

from stable_baselines3 import PPO
from trading.environments.forex_env2_flat import ForexTradingEnv
from concurrent.futures import ThreadPoolExecutor
from apscheduler.schedulers.background import BackgroundScheduler
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments
from oandapyV20 import API
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import logging
import threading
import time
from pathlib import Path

from google.cloud import storage
from google.oauth2 import service_account

client = storage.Client(credentials=credentials)

# Initialize IndicatorManager and DataProcessor
indicator_manager = IndicatorManager()
data_processor = DataPreprocessor()

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('deploy.py')

# OANDA API setup
# Ensure that API credentials are loaded securely, e.g., from environment variables
account_id = '101-004-30348600-001'
OANDA_API_KEY = '9317ace4596d61e3e98b1a53b2342483-45d3ad4084c80b111727a9fada9ef0ff'
OANDA_ACCOUNT_ID = account_id
# OANDA_ACCOUNT_ID = '101-004-30348600-002'
OANDA_ENV = 'practice'  # or 'practice'

# Initialize OANDA API client
client = API(access_token=OANDA_API_KEY, environment=OANDA_ENV)

# Currency pairs
currency_pairs = {
    'EUR_USD': 94_510.0,
    # 'GBP_USD': 78_500.0,
    # 'USD_JPY': 100_000.0,
    # 'USD_CHF': 100_000.0,
    # 'USD_CAD': 100_000.0,
    # 'AUD_USD': 153_000.0,
    # 'NZD_USD': 171_430.0,

    # # Cross Pairs
    # 'EUR_GBP': 94_510,
    # 'EUR_CHF': 94_510,
    # 'EUR_JPY': 94_510,
    # 'EUR_CAD': 94_510,
    # 'GBP_CHF': 78_500.0,
    # 'GBP_JPY': 78_500.0,
    # 'CHF_JPY': 88_100.0,
    # 'AUD_JPY': 153_000.0,
    # 'NZD_JPY': 171_430.0,

    # # Precious Metals
    # 'XAU_USD': 37.68,  # Gold
    # 'XAG_USD': 3_266  # Silver
}


# Global variables for data and positions
data_storage = {}
positions_storage = {}
data_lock = threading.Lock()
positions_lock = threading.Lock()



# Improved data processing pipeline with clean separation of concerns

from pathlib import Path
from queue import Queue
import threading
import pandas as pd
import numpy as np
from typing import Dict, Optional
import logging

class FastDataManager:
    """
    High-performance data manager optimized for low-latency trading.
    Keeps data in memory for fast access while handling persistence asynchronously.
    """
    
    def __init__(
        self,
        base_storage_path: str,
        max_history_size: int = 10000  # Number of candlesticks to keep
    ):
        self.base_storage_path = Path(base_storage_path)
        self.max_history_size = max_history_size
        
        # Separate storage for different data types to optimize access patterns
        self.raw_data: Dict[str, pd.DataFrame] = {}        # OHLCV + indicators
        self.normalized_data: Dict[str, pd.DataFrame] = {} # Normalized for predictions
        
        # Thread safety mechanisms
        self.data_lock = threading.Lock()
        self.save_queue = Queue()
        
        # Initialize components
        self.indicator_manager = IndicatorManager()
        self.data_processor = DataPreprocessor()
        
        # Start background save worker
        self.save_worker = threading.Thread(
            target=self._parquet_save_worker,
            daemon=True,
            name="ParquetSaveWorker"
        )
        self.save_worker.start()
        
        self.logger = logging.getLogger(__name__)

    def initialize_pair(self, pair: str) -> bool:
        """
        Load initial data for a pair into memory and prepare it for trading.
        Returns True if initialization was successful.
        """
        try:
            # Load historical data
            parquet_path = self.base_storage_path / f"{pair}_5T_indics_1H.parquet"
            df = pd.read_parquet(parquet_path)
            
            # Ensure proper timezone
            if df.index.tz is None:
                df.index = df.index.tz_localize('UTC')
            
            # Trim to maximum history size if needed
            if len(df) > self.max_history_size:
                df = df.iloc[-self.max_history_size:]
            
            with self.data_lock:
                # Store both raw and normalized versions
                self.raw_data[pair] = df
                self.normalized_data[pair] = self.data_processor.normalize_simple(df)
                
            self.logger.info(f"Initialized data for {pair}, loaded {len(df)} candles")
            return True
            
        except Exception as e:
            self.logger.error(f"Failed to initialize {pair}: {str(e)}")
            return False

    def update_pair_data(self, pair: str) -> bool:
        """
        Fast update of pair data focusing on speed of execution.
        Returns True if new data was added and predictions should be made.
        """
        try:
            with self.data_lock:
                if pair not in self.raw_data:
                    raise KeyError(f"Pair {pair} not initialized")
                    
                df = self.raw_data[pair]
                last_timestamp = df.index[-1]

            current_time = pd.Timestamp.now(tz='UTC')
            
            # Check if update is needed (5-minute intervals)
            if current_time - last_timestamp >= timedelta(minutes=5):
                # Fetch new data from OANDA
                new_data = self.fetch_missing_candles(pair, last_timestamp)
                
                if not new_data.empty:
                    with self.data_lock:
                        # Update raw data
                        combined_df = pd.concat([self.raw_data[pair], new_data])
                        combined_df = combined_df[~combined_df.index.duplicated(keep='last')]
                        combined_df.sort_index(inplace=True)
                        
                        # Trim to max history size
                        if len(combined_df) > self.max_history_size:
                            combined_df = combined_df.iloc[-self.max_history_size:]
                        
                        # Calculate technical indicators
                        combined_df_with_indicators = self.indicator_manager.calculate_indicators(combined_df)
                        
                        # Update both storage types
                        self.raw_data[pair] = combined_df_with_indicators
                        self.normalized_data[pair] = self.data_processor.normalize_simple(
                            combined_df_with_indicators
                        )
                        
                        # Queue asynchronous save
                        self.save_queue.put((pair, combined_df_with_indicators))
                        
                        self.logger.debug(
                            f"Updated {pair} with {len(new_data)} new candles. "
                            f"Total length: {len(combined_df)}"
                        )
                    return True
            
            return False

        except Exception as e:
            self.logger.error(f"Error updating data for {pair}: {str(e)}")
            raise

    def get_prediction_data(self, pair: str, sequence_length: int) -> np.ndarray:
        """
        Get normalized data sequence for prediction with minimal latency.
        Raises KeyError if pair data is not available.
        """
        with self.data_lock:
            if pair not in self.normalized_data:
                raise KeyError(f"No data available for {pair}")
            
            df = self.normalized_data[pair]
            
            if len(df) < sequence_length:
                raise ValueError(
                    f"Insufficient data for {pair}. "
                    f"Need {sequence_length} samples, have {len(df)}"
                )
            
            # Get the latest sequence and prepare it for the model
            sequence = df.iloc[-sequence_length:].values
            return sequence.T.flatten()

    def _parquet_save_worker(self) -> None:
        """Background worker for asynchronous parquet file saves."""
        while True:
            try:
                pair, df = self.save_queue.get()
                if pair is None:  # Shutdown signal
                    break
                    
                parquet_path = self.base_storage_path / f"{pair}_5T_indics_1H.parquet"
                df.to_parquet(parquet_path)
                
                self.logger.debug(f"Saved parquet file for {pair}")
                
            except Exception as e:
                self.logger.error(f"Error saving parquet file: {str(e)}")
            finally:
                self.save_queue.task_done()

    def shutdown(self) -> None:
        """
        Clean shutdown ensuring all pending saves are completed.
        Should be called before program exit.
        """
        self.logger.info("Initiating data manager shutdown...")
        
        # Signal save worker to stop
        self.save_queue.put((None, None))
        
        # Wait for remaining saves with timeout
        try:
            self.save_queue.join()
            self.save_worker.join(timeout=30)  # 30 second timeout
            self.logger.info("Data manager shutdown complete")
        except TimeoutError:
            self.logger.warning("Shutdown timed out waiting for saves to complete")

def initialize_trading_system():
    """Initialize the complete trading system with fast data handling."""
    # Initialize data manager
    data_manager = FastDataManager(
        base_storage_path="/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes",
        max_history_size=10000
    )
    
    # Load models
    models = {}
    for pair in currency_pairs:
        try:
            # Initialize data first
            if not data_manager.initialize_pair(pair):
                logger.error(f"Failed to initialize data for {pair}, skipping...")
                continue
                
            # Load model and environment
            model_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/models_and_vecs/{pair}_best_model'
            env_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/models_and_vecs/{pair}_vec_normalize.pkl'
            
            # Create environment with normalized data
            vec_env = DummyVecEnv([lambda: ForexTradingEnv(
                data_manager.raw_data[pair], pair
            )])
            
            env = VecNormalize.load(env_path, vec_env)
            env.training = False
            env.norm_reward = False
            
            # Load the model
            model = PPO.load(model_path, env=env)
            models[pair] = model
            
            logger.info(f"Initialized model for {pair}")
            
        except Exception as e:
            logger.error(f"Error initializing {pair}: {str(e)}")
            continue
    
    return data_manager, models

def run_trading_cycle(data_manager: FastDataManager, models: Dict[str, PPO]):
    """Execute a single trading cycle with minimal latency."""
    logger.info("Starting trading cycle")
    
    for pair in currency_pairs:
        try:
            # Fast update check
            if data_manager.update_pair_data(pair):
                # Get prediction data (fast in-memory access)
                sequence = data_manager.get_prediction_data(pair, sequence_length=5)
                
                # Reshape for model
                obs_array = sequence.reshape((1, -1))
                
                # Get prediction
                model = models[pair]
                obs_array = model.env.normalize_obs(obs_array)
                action, _ = model.predict(obs_array, deterministic=True)
                
                # Execute trade if needed
                current_position = positions_storage.get(pair, 'NO_POSITION')
                action_name = {0: 'NO_POSITION', 1: 'LONG', 2: 'SHORT'}[action[0]]
                
                if current_position != action_name:
                    execute_trade(pair, current_position, action_name)
        
        except Exception as e:
            logger.error(f"Error in trading cycle for {pair}: {str(e)}")

def execute_trade(pair: str, current_position: str, new_position: str):
    """Execute trade with position management."""
    try:
        # Close existing position if any
        if current_position != 'NO_POSITION':
            close_position(pair, current_position)
        
        # Open new position if not moving to neutral
        if new_position != 'NO_POSITION':
            open_position(pair, new_position)
        
        # Update position storage
        with positions_lock:
            positions_storage[pair] = new_position
            
        logger.info(f"Executed trade for {pair}: {current_position} -> {new_position}")
        
    except Exception as e:
        logger.error(f"Error executing trade for {pair}: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        # Initialize system
        data_manager, models = initialize_trading_system()
        
        # Set up scheduler
        scheduler = BackgroundScheduler()
        scheduler.add_job(
            lambda: run_trading_cycle(data_manager, models),
            'cron',
            minute='*/5',
            second=0
        )
        scheduler.start()
        
        # Keep the main thread alive
        try:
            while True:
                time.sleep(60)
        except (KeyboardInterrupt, SystemExit):
            logger.info("Shutting down trading system...")
            scheduler.shutdown()
            data_manager.shutdown()
            logger.info("Trading system shutdown complete")
            
    except Exception as e:
        logger.error(f"Fatal error in trading system: {str(e)}")
        raise

####! OLD
###################################
###################################
###################################
###################################

class DataManager:
    """Manages data loading, updating, and processing with clear separation of raw and normalized data."""
    
    def __init__(self, base_storage_path: str = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm'):
        self.base_storage_path = Path(base_storage_path)
        self.data_storage = {}  # In-memory storage for normalized data
        self.data_lock = threading.Lock()
        self.indicator_manager = IndicatorManager()
        self.data_processor = DataPreprocessor()
        
    def fetch_missing_candles(self, pair: str, last_timestamp: pd.Timestamp) -> pd.DataFrame:
        """Fetch new candles from OANDA starting from last known timestamp."""
        params = {
            "from": last_timestamp.strftime('%Y-%m-%dT%H:%M:%SZ'),
            "granularity": "M5",
            "price": "M"
        }
        
        r = instruments.InstrumentsCandles(instrument=pair, params=params)
        response = client.request(r)
        candles = response.get('candles', [])
        
        if not candles:
            return pd.DataFrame()
            
        df_list = [{
            'timestamp': pd.to_datetime(candle['time']),
            'open': float(candle['mid']['o']),
            'high': float(candle['mid']['h']),
            'low': float(candle['mid']['l']),
            'close': float(candle['mid']['c']),
            # 'volume': int(candle['volume'])
        } for candle in candles if candle['complete']]
        
        if not df_list:
            return pd.DataFrame()
            
        df = pd.DataFrame(df_list)
        df.set_index('timestamp', inplace=True)
        df.index = df.index.tz_localize('UTC')
        return df

    def update_pair_data(self, pair: str) -> None:
        """Update data for a single pair, maintaining both parquet and in-memory states."""
        try:
            # Load existing parquet data
            parquet_path = self.base_storage_path / f"{pair}_5T_indics_1H_not_norm.parquet"
            df = pd.read_parquet(parquet_path)
        
            # Ensure proper timezone
            if df.index.tz is None:
                df.index = df.index.tz_localize('UTC')
            
            # Check for missing data
            last_timestamp = df.index[-1]
            current_time = pd.Timestamp.now(tz='UTC')
            
            if current_time - last_timestamp >= timedelta(minutes=5):
                # Fetch new data
                new_data = self.fetch_missing_candles(pair, last_timestamp)
                
                if not new_data.empty:
                    # First combine the data
                    combined_df = pd.concat([df, new_data])
                    combined_df = combined_df[~combined_df.index.duplicated(keep='last')]
                    combined_df.sort_index(inplace=True)
                    
                    # Now calculate indicators on the entire dataset
                    combined_df_with_indicators = self.indicator_manager.calculate_indicators(combined_df)
                    
                    # Save updated raw data with indicators
                    combined_df_with_indicators.to_parquet(parquet_path)
                    
                    # Update in-memory normalized data
                    normalized_df = self.data_processor.normalize_simple(combined_df_with_indicators)
                    
                    with self.data_lock:
                        self.data_storage[pair] = normalized_df
                        
                    logger.info(f"Updated data for {pair}: {len(new_data)} new candles")
                else:
                    logger.info(f"No new data available for {pair}")
            else:
                # If no new data needed, just ensure normalized version is in memory
                if pair not in self.data_storage:
                    normalized_df = self.data_processor.normalize_simple(df)
                    with self.data_lock:
                        self.data_storage[pair] = normalized_df
        
        except Exception as e:
            logger.error(f"Error updating data for {pair}: {str(e)}")
            raise

    def get_data_for_prediction(self, pair: str, sequence_length: int) -> np.ndarray:
        """Get normalized data sequence for model prediction."""
        with self.data_lock:
            if pair not in self.data_storage:
                raise KeyError(f"No data available for {pair}")
            df = self.data_storage[pair]
            
        sequence = df.iloc[-sequence_length:].values
        return sequence.T.flatten()  # Return flattened sequence in correct format

def initialize_data():
    """Initialize data manager and load initial data."""
    data_manager = DataManager('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm')
    
    for pair in currency_pairs:
        try:
            data_manager.update_pair_data(pair)
            logger.info(f"Initialized data for {pair}")
        except Exception as e:
            logger.error(f"Failed to initialize {pair}: {e}")
    
    return data_manager

# Load models and normalization parameters
models = {}
for pair in currency_pairs:
    # Load the model
    model_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/models_and_vecs/{pair}_best_model'
    env_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/models_and_vecs/{pair}_vec_normalize.pkl'

    # Load real data for this pair from your storage
    df = pd.read_parquet(f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/{pair}_5T_indics_1H_norm.parquet')
    

    
    vec_env = DummyVecEnv([lambda: ForexTradingEnv(df, pair)])
    env = VecNormalize.load(env_path, vec_env)
    # Create a dummy environment and set the statistics
 
    env = VecNormalize(env)
    # env.obs_rms = vec_normalize.obs_rms
    # env.ret_rms = vec_normalize.ret_rms
    env.training = False
    env.norm_reward = False

    # Load the model
    model = PPO.load(model_path, env=env)
    models[pair] = model


def calculate_indicators(df):
    print("Calculating indicators...")
    # Resample 5-minute data to 1-hour data for indicator calculation
    df_hourly = df.resample('1H').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
    }).dropna()

    # Calculate indicators on 1-hour data using IndicatorManager
    indicator_df = indicator_manager.calculate_indicators(
        df_hourly,
        selected_indicators=None,  # Calculate all indicators
        indicator_timeframe=None   # Indicators are calculated on the resampled data
    )

    # Forward-fill the indicators back to the 5-minute DataFrame
    df = df.merge(indicator_df, left_index=True, right_index=True, how='left')
    df.fillna(method='ffill', inplace=True)

    # Drop rows with NaN values (if any remain)
    df.dropna(inplace=True)
    print("Indicators calculated.")

    return df


def normalize_data(df):
    print("Normalizing data...")
    # Use DataProcessor to normalize data
    df_normalized = data_processor.normalize_simple(df)
    return df_normalized


def load_data_from_gcs(pair):
    # Initialize GCS client
    storage_client = storage.Client()
    bucket_name = 'fx_data_sets_15min_1h_tech_norm'  # Replace with your bucket name
    bucket = storage_client.bucket(bucket_name)
    #! change name below
    blob = bucket.blob(f'{pair}_data.parquet')
    temp_file = f'/tmp/{pair}_data.parquet'
    blob.download_to_filename(temp_file)
    df = pd.read_parquet(temp_file)
    # Ensure that timestamp is the index and is timezone-aware
    if df.index.tz is None:
        df.index = df.index.tz_localize('UTC')
    print(f"Loaded data for {pair} from GCS.")
    print(df.head())
    return df


def fetch_missing_candles(pair, last_timestamp):
    # Fetch missing candles from OANDA API starting from last_timestamp
    print(f"Fetching missing candles for {pair} from {last_timestamp}...")
    client = API(access_token=OANDA_API_KEY, environment='practice')
    params = {
        "from": last_timestamp.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "granularity": "M5",  # 5-minute candles
        "price": "M",  # Midpoint prices
    }
    r = instruments.InstrumentsCandles(instrument=pair, params=params)
    response = client.request(r)
    candles = response.get('candles', [])
    # client.request(r)
    # candles = r.response.get('candles', [])
    df_list = []
    print(f"Fetched {len(candles)} candles for {pair}.")
    for candle in candles:
        if candle['complete']:
            time_candle = pd.to_datetime(candle['time'])
            open_price = float(candle['mid']['o'])
            high_price = float(candle['mid']['h'])
            low_price = float(candle['mid']['l'])
            close_price = float(candle['mid']['c'])
            volume = int(candle['volume'])
            df_list.append({
                'timestamp': time_candle,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'volume': volume,
            })
    if df_list:
        df_new = pd.DataFrame(df_list)
        df_new.set_index('timestamp', inplace=True)
        return df_new
    else:
        return pd.DataFrame()


def fetch_current_positions():
    print("Fetching current positions...")
    r = OpenPositions(accountID=OANDA_ACCOUNT_ID)
    response = client.request(r)
    open_positions = response['positions']
    print(f"Current positions: {open_positions}")
    with positions_lock:
        positions_storage.clear()
        for pos in open_positions:
            pair = pos['instrument']
            if pair in currency_pairs:
                long_units = float(pos.get('long', {}).get('units', 0))
                short_units = float(pos.get('short', {}).get('units', 0))
                if long_units > 0:
                    positions_storage[pair] = 'LONG'
                elif short_units < 0:
                    positions_storage[pair] = 'SHORT'
                else:
                    positions_storage[pair] = 'NO_POSITION'
            else:
                positions_storage[pair] = 'NO_POSITION'
    print(f"Current positions: {positions_storage}")

def fetch_and_update_data(pair):
    with data_lock:
        df = data_storage.get(pair)

    if df is None:
        # Load existing dataset from GCS
        df = load_data_from_gcs(pair)
        print(f"Loaded existing data for {pair} from GCS.")
        print(f'df head from GCS: {df.head()}')
    # Check last available candle
    last_timestamp = df.index[-1]
    current_time = datetime.now(datetime.timezone.utc()).replace(
        tzinfo=pd.Timestamp.utcnow().tz)
    if current_time - last_timestamp >= timedelta(minutes=5):
        # Fetch missing candles
        df_new = fetch_missing_candles(pair, last_timestamp)
        if not df_new.empty:
            df = pd.concat([df, df_new])
            df = df[~df.index.duplicated(keep='last')]
            df.sort_index(inplace=True)
            # Calculate technical indicators on the entire DataFrame
            df = calculate_indicators(df)
            # Normalize data
            df = normalize_data(df)
            # Update data storage
            with data_lock:
                print(f"Updating data for {pair} in memory.")
                data_storage[pair] = df
        else:
            logger.info(f"No new data for {pair}")
    else:
        logger.info(f"No missing data for {pair}")


def update_data():
    print("Updating data...")
    for pair in currency_pairs.keys():
        try:
            fetch_and_update_data(pair)
            logger.info(f"Data updated for {pair}")
        except Exception as e:
            logger.error(f"Error updating data for {pair}: {e}")


def predict_and_trade(pair):
    with data_lock:
        df = data_storage[pair]
    # Prepare the sequence data
    sequence_length = 5  # Adjust based on your model
    obs = df.iloc[-sequence_length:]
    # Convert to numpy array
    obs_array = obs.values  # Shape: (sequence_length, num_features)
    # Flatten the observation if needed
    obs_array = obs_array.T.flatten()
    # Reshape to (1, observation_space)
    obs_array = obs_array.reshape((1, -1))
    # Get the model
    model = models[pair]
    # Normalize the observation
    obs_array = model.env.normalize_obs(obs_array)
    # Predict
    action, _ = model.predict(obs_array)
    # Map action to position
    action_mapping = {0: 'NO_POSITION', 1: 'LONG', 2: 'SHORT'}
    action_name = action_mapping.get(action[0])
    with positions_lock:
        current_position = positions_storage.get(pair, 'NO_POSITION')
    if current_position != action_name:
        # Execute trade via OANDA API
        # Close existing position if any
        if current_position != 'NO_POSITION':
            close_position(pair, current_position)
        # Open new position if action_name is not NO_POSITION
        if action_name != 'NO_POSITION':
            open_position(pair, action_name)
        with positions_lock:
            positions_storage[pair] = action_name
        logger.info(f"Executed action {action_name} for {pair}")
    else:
        logger.info(
            f"No action needed for {pair}, current position is {current_position}")


def open_position(pair, action_name):
    # Define the order data
    units = currency_pairs[pair]  # Get the notional amount for this pair
    if action_name == 'LONG':
        units = units  # Positive units for buy
    elif action_name == 'SHORT':
        units = -units  # Negative units for sell
    else:
        return
    data = {
        "order": {
            "instrument": pair,
            "units": str(units),
            "type": "MARKET",
            "positionFill": "DEFAULT"
        }
    }
    r = orders.OrderCreate(accountID=OANDA_ACCOUNT_ID, data=data)
    client.request(r)
    logger.info(f"Opened {action_name} position for {pair}")


def close_position(pair, current_position):
    data = {}
    if current_position == 'LONG':
        data = {
            "longUnits": "ALL"
        }
    elif current_position == 'SHORT':
        data = {
            "shortUnits": "ALL"
        }
    else:
        return
    r = positions.PositionClose(
        accountID=OANDA_ACCOUNT_ID, instrument=pair, data=data)
    client.request(r)
    logger.info(f"Closed {current_position} position for {pair}")


def sync_positions():
    print("Syncing positions with OANDA API")
    fetch_current_positions()
    logger.info("Positions synchronized with OANDA API")


def process_pair(pair):
    try:
        predict_and_trade(pair)
    except Exception as e:
        logger.error(f"Error processing {pair}: {str(e)}")


def run_trading_cycle():
    logger.info("Starting trading cycle")
    # Update data first
    update_data()
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(process_pair, currency_pairs.keys())
    logger.info("Trading cycle completed")


def initialize():
    # Initialize data storage
    for pair in currency_pairs.keys():
        try:
            # Load existing dataset from GCS
            df = load_data_from_gcs(pair)
            # Ensure that timestamp is the index and is timezone-aware
            if df.index.tz is None:
                df.index = df.index.tz_localize('UTC')
            # Check for missing data and fetch if necessary
            fetch_and_update_data(pair)
            # Calculate technical indicators on the entire DataFrame
            df = calculate_indicators(df)
            # Normalize data
            df = normalize_data(df)
            # Update data storage
            with data_lock:
                data_storage[pair] = df
            logger.info(f"Initialized data for {pair}")
        except Exception as e:
            logger.error(f"Error initializing data for {pair}: {e}")
    # Synchronize positions at startup
    sync_positions()





In [ ]:
try:
    # Initialize system
    data_manager, models = initialize_trading_system()
    
    # Set up scheduler
    scheduler = BackgroundScheduler()
    scheduler.add_job(
        lambda: run_trading_cycle(data_manager, models),
        'cron',
        minute='*/5',
        second=0
    )
    scheduler.start()
    
    # Keep the main thread alive
    try:
        while True:
            time.sleep(60)
    except (KeyboardInterrupt, SystemExit):
        logger.info("Shutting down trading system...")
        scheduler.shutdown()
        data_manager.shutdown()
        logger.info("Trading system shutdown complete")
        
except Exception as e:
    logger.error(f"Fatal error in trading system: {str(e)}")
    raise

In [ ]:
initialize()
# Scheduler setup
scheduler = BackgroundScheduler()
scheduler.add_job(run_trading_cycle, 'cron', minute='*/5', second=0)
scheduler.start()

# Keep the main thread alive
try:
    while True:
        time.sleep(60)
except (KeyboardInterrupt, SystemExit):
    scheduler.shutdown()
    logger.info("Scheduler shut down")